In [1]:
# Import findspark and initialize.
import os # allow for pyspark / hadoop / java instance components to be installed into the runtime
# set up a reference to the pyspark instance - stable version - 3.3.2
spark_version = 'spark-3.3.2'
# use os.environ function to reference 'SPARK_VERSION' property
os.environ['SPARK_VERSION'] = spark_version
# Install the spark and java components
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark
# set the runtime environment variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64" # sets the path to the Java Virtual Machine (Java Development Kit)
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"
# start a spark session by importing and using findspark
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 336 kB in 3s (125 kB/s)
Reading package lists... Done


In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the data from the url
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

# add the file to the spark session
spark.sparkContext.addFile(url)

# read in the AWS S3 bucket into a DataFrame.
homeSalesDF = spark.read.csv(
    SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, inferSchema=True
)

homeSalesDF.show(5)

+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|               date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08 00:00:00|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13 00:00:00|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12 00:00:00|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16 00:00:00|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08 00:00:00|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+-------

In [4]:
# print schema and check data types
homeSalesDF.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [5]:
# 2. Create a temporary view of the DataFrame: sales_view
homeSalesDF.createOrReplaceTempView("sales_view")


In [6]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
queryString1 = """
(
  SELECT year(date) AS year_sold, ROUND(AVG(price), 2) AS average_price
  FROM sales_view
  WHERE bedrooms = 4
  GROUP BY year(date)
)
"""
spark.sql(queryString1).show()

+---------+-------------+
|year_sold|average_price|
+---------+-------------+
|     2022|    296363.88|
|     2019|     300263.7|
|     2020|    298353.78|
|     2021|    301819.44|
+---------+-------------+



In [7]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
queryString2 = """
(
  SELECT date_built AS year_built, ROUND(AVG(price), 2) AS average_price
  FROM sales_view
  WHERE bedrooms = 3 AND bathrooms = 3
  GROUP BY date_built
)
"""
spark.sql(queryString2).show()

+----------+-------------+
|year_built|average_price|
+----------+-------------+
|      2015|     288770.3|
|      2013|    295962.27|
|      2014|    290852.27|
|      2012|    293683.19|
|      2016|    290555.07|
|      2010|    292859.62|
|      2011|    291117.47|
|      2017|    292676.79|
+----------+-------------+



In [8]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
queryString3 = """
(
  SELECT date_built AS year_built, ROUND(AVG(price), 2) AS average_price
  FROM sales_view
  WHERE bedrooms = 3 AND bathrooms = 3 AND floors=2 AND sqft_living >= 2000
  GROUP BY date_built
)
"""
spark.sql(queryString3).show()


+----------+-------------+
|year_built|average_price|
+----------+-------------+
|      2015|    297609.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2012|    307539.97|
|      2016|     293965.1|
|      2010|    285010.22|
|      2011|    276553.81|
|      2017|    280317.58|
+----------+-------------+



In [9]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
start_time = time.time()

queryString4 = """
(
  SELECT view, ROUND(AVG(price), 2) AS average_price
  FROM sales_view
  WHERE price >= 350000
  GROUP BY view
  ORDER BY view
)
"""
spark.sql(queryString4).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|   0|    403848.51|
|   1|    401044.25|
|   2|    397389.25|
|   3|     398867.6|
|   4|    399631.89|
|   5|    401471.82|
|   6|    395655.38|
|   7|    403005.77|
|   8|    398592.71|
|   9|    401393.34|
|  10|    401868.43|
|  11|    399548.12|
|  12|    401501.32|
|  13|    398917.98|
|  14|    398570.03|
|  15|     404673.3|
|  16|    399586.53|
|  17|    398474.49|
|  18|    399332.91|
|  19|    398953.17|
+----+-------------+
only showing top 20 rows

--- 0.9915339946746826 seconds ---


In [10]:
# 7. Cache the temporary table sales_view.
spark.catalog.cacheTable("sales_view")

In [11]:
# 8. Check if the table is cached.
spark.catalog.isCached('sales_view')

True

In [12]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

queryString5 = """
(
  SELECT view, ROUND(AVG(price), 2) AS average_price
  FROM sales_view
  WHERE price >= 350000
  GROUP BY view
  ORDER BY view
)
"""
spark.sql(queryString5).show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|average_price|
+----+-------------+
|   0|    403848.51|
|   1|    401044.25|
|   2|    397389.25|
|   3|     398867.6|
|   4|    399631.89|
|   5|    401471.82|
|   6|    395655.38|
|   7|    403005.77|
|   8|    398592.71|
|   9|    401393.34|
|  10|    401868.43|
|  11|    399548.12|
|  12|    401501.32|
|  13|    398917.98|
|  14|    398570.03|
|  15|     404673.3|
|  16|    399586.53|
|  17|    398474.49|
|  18|    399332.91|
|  19|    398953.17|
+----+-------------+
only showing top 20 rows

--- 1.86234450340271 seconds ---


In [14]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
homeSalesDF.write.partitionBy("date_built").parquet("formatted_sales_view_partitioned.parquet")

In [15]:
# 11. Read the formatted parquet data.
formattedHomeSalesDF = spark.read.parquet("formatted_sales_view_partitioned.parquet")
formattedHomeSalesDF

DataFrame[id: string, date: timestamp, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int, date_built: int]

In [16]:
# 12. Create a temporary table for the parquet data.
formattedHomeSalesDF.createOrReplaceTempView("parquet_home_sales")

In [17]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

parquetQuery = """
  SELECT view, ROUND(AVG(price), 2) AS average_price
  FROM parquet_home_sales
  WHERE price >= 350000
  GROUP BY view
  ORDER BY view
"""

spark.sql(parquetQuery).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|   0|    403848.51|
|   1|    401044.25|
|   2|    397389.25|
|   3|     398867.6|
|   4|    399631.89|
|   5|    401471.82|
|   6|    395655.38|
|   7|    403005.77|
|   8|    398592.71|
|   9|    401393.34|
|  10|    401868.43|
|  11|    399548.12|
|  12|    401501.32|
|  13|    398917.98|
|  14|    398570.03|
|  15|     404673.3|
|  16|    399586.53|
|  17|    398474.49|
|  18|    399332.91|
|  19|    398953.17|
+----+-------------+
only showing top 20 rows

--- 0.9681723117828369 seconds ---


In [18]:
# 14. Uncache the sales_view temporary table.
spark.catalog.uncacheTable("sales_view")

In [19]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('sales_view')


False